In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import math 
import copy
import random
import argparse
import torch.optim as optim



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
def load_data(dataset_name, K):
    if dataset_name == "statlog":
        ##statlog
        __context_size__ = 8
        __arm_size__ = 7
        fr = open("shuttle.trn",'r+')
        CONTEXT = []
        REWARD = []
        SYM = {}
        aa = 0
        K = 0
        for line in fr:
            K+=1
            aaa = line.split("\n")
            aaa = aaa[0].split(" ")
            temp = np.int64(aaa[9])-1
            context = np.double(aaa[1:9])
            context = context/np.linalg.norm(context)
            ttt = context
            CONTEXT.append(ttt)
            reward = np.zeros(__arm_size__)
            reward[temp] = 1
            REWARD.append(reward)

    elif dataset_name == "magic":
        ####magic gamma

        __context_size__ = 10
        __arm_size__ = 2
        fr = open("letter.data", 'r+')

        K = 0
        CONTEXT = []
        REWARD = []

        fr.close()
        fr = open("magic04.data", 'r+')
        for line in fr:

            context = []
            aaa = line.split(",")
            context = np.double(aaa[0:__context_size__])
            context = context / np.linalg.norm(context)
            K += 1
            CONTEXT.append(np.array(context))
            __context_size__ = len(context)
            reward = np.zeros(__arm_size__)
            if aaa[10] == 'g\n':
                reward[0] = 1
            else:
                reward[1] = 1
            REWARD.append(reward)

        fr.close()
    elif dataset_name == "covertype":
        ##For covertypr
        __context_size__ = 14
        __arm_size__ = 7
        fr = open("covtype.data", 'r+')
        CONTEXT = []
        REWARD = []
        SYM = {}
        aa = 0
        K = 0
        for line in fr:
            K += 1
            aaa = line.split(",")
            __context_size__ = len(aaa) - 1
            temp = int(aaa[__context_size__]) - 1
            context = np.double(aaa[0:__context_size__])
            context = context / np.linalg.norm(context)
            ttt = context
            CONTEXT.append(ttt)
            reward = np.zeros(__arm_size__)
            reward[temp] = 1
            REWARD.append(reward)
    else:
        assert('dataset not avaiable')

    TEMP_CONTEXT = []
    TEMP_REWARD = []
    sli = np.random.permutation(K)
    for i in range(K):
        TEMP_CONTEXT.append(CONTEXT[sli[i]])
        TEMP_REWARD.append(REWARD[sli[i]])
    CONTEXT = TEMP_CONTEXT
    REWARD = TEMP_REWARD

    return __context_size__, __arm_size__, CONTEXT, REWARD, sli

In [3]:
# define the hyperparameters
input_size = 15
hidden_size = 50
output_size = 15
num_layers = 10
arm_size = 7
beta = 0.1
lambd = 1
H = 100

In [4]:
# initialize the parameters

theta = np.random.randn(output_size, 1) / np.sqrt(output_size)
theta = torch.tensor(theta, dtype = torch.float32)

LAMBDA = lambd * torch.eye(output_size, dtype=torch.float32)
bb = torch.zeros(LAMBDA.size()[0], dtype=torch.float).view(-1, 1)

In [5]:
# create the model 
import modeldefine
import torch.nn as nn
model = modeldefine.Model(input_size, hidden_size, output_size, num_layers).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

class CustomMSELoss(nn.Module):
    def __init__(self, theta):
        super(CustomMSELoss, self).__init__()
        self.theta = theta
        self.mse_loss = nn.MSELoss()

    def forward(self, output, target):
        prediction = torch.sum(output * self.theta, dim=1)  # 按元素乘法并求和
        return self.mse_loss(prediction, target)
print(model)

tensor([15, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 15], dtype=torch.int32)
Model(
  (layers): ModuleList(
    (0): Linear(in_features=30, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): ReLU()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): ReLU()
    (8): Linear(in_features=100, out_features=100, bias=True)
    (9): ReLU()
    (10): Linear(in_features=100, out_features=100, bias=True)
    (11): ReLU()
    (12): Linear(in_features=100, out_features=100, bias=True)
    (13): ReLU()
    (14): Linear(in_features=100, out_features=100, bias=True)
    (15): ReLU()
    (16): Linear(in_features=100, out_features=100, bias=True)
    (17): ReLU()
    (18): Linear(in_features=100, out_features=100, bias=True)
    (19): ReLU()
    (20): Linear(in_features=100, out_features=15, bias=True)
    (21): ReLU()
  )
)


In [6]:
def TRANS(c, a, arm_size):
    #### transfer an array context + action to new context with dimension 2*(__context__ + __armsize__)
    dim = len(c)
    action = np.zeros(arm_size)
    action[a] = 1
    c_final = np.append(c, action)
    c_final = torch.from_numpy(c_final).to(device)
    c_final = c_final.view((len(c_final), 1))
    c_final = c_final.repeat(2, 1)
    c_final = torch.tensor(c_final, dtype=torch.float32).view(1,-1)
    return c_final

def UCB(A, phi):
    #### ucb term
    phi = phi.view(-1,1)
    try:
        tmp, LU = torch.linalg.solve(phi,A)
    except:
        A = A.detach().numpy()
        phi2 = phi.detach().numpy()
        tmp = torch.Tensor(np.linalg.solve(A, phi2))

    return torch.sqrt(torch.matmul(torch.transpose(phi,1,0), tmp))

def get_action(model, context_info, random_indicator):
    ucb = [] 
    bphi = []
    for i in range(arm_size):
        temp = TRANS(context_info, i, arm_size).to(device)
        bphi.append(temp.cpu())
        feature = model(temp).cpu()

        first_item = torch.mm(feature, theta)
        ucb_item = beta * UCB(LAMBDA, feature)
        ucb.append(first_item + ucb_item)
    
    if random_indicator == 0:
        return ucb.index(max(ucb)), bphi[ucb.index(max(ucb))] 
    else:
        num =  np.random.randint(0, arm_size)
        return num, bphi[num]
    
def update_lambda_bb(lambd, bb, context_info, action,reward):
    temp = TRANS(context_info, action, arm_size).to(device)
    feature = model(temp).cpu()
    templambda = lambd + torch.mm(feature.t(), feature)
    tempbb = bb + reward * feature.t()
    return templambda, tempbb

def update_theta(lambd, bb):
    return torch.linalg.solve(lambd, bb)

In [7]:
THETA_action = []
CONTEXT_action = []
REWARD_action = []
context_size, arm_size, CONTEXT_list, REWARD, sli = load_data("statlog", 15000)

for t in range(0,10000):
    context = CONTEXT_list[t]
    if t <= 21:
        action, c_act = get_action(model, context, 1)
    else: 
        action, c_act = get_action(model, context, 0) 

    # get the reward
    reward = REWARD[t][action] 

    # construct the training set for neural network
    if t % H == 0:
        CONTEXT_action = []
        REWARD_action = []
        CONTEXT_action = c_act
        REWARD_action = torch.tensor([reward],device= device, dtype=torch.float32)
    else:
        CONTEXT_action = torch.cat((CONTEXT_action, c_act), 0)
        REWARD_action = torch.cat((REWARD_action, torch.tensor([reward],device= device, dtype=torch.float32)), 0)

    # update lambda, bb and theta
    LAMBDA, bb = update_lambda_bb(LAMBDA,bb,context, action, reward)
    theta = update_theta(LAMBDA, bb)

    if t % H == 0:
        THETA_action = []
        THETA_action = theta.view(1,-1)
    else: 
        THETA_action = torch.cat((THETA_action, theta.view(1,-1)), 0)
    
    if (t % H == (H -1)):
        # update the model
        loss_function = CustomMSELoss(THETA_action.detach().to(device))

        epochs = 10
        for epoch in range(epochs):
            # 前向传播
            outputs = model(CONTEXT_action.to(device))
            loss = loss_function(outputs, REWARD_action)
            
            # 打印损失
            if epoch == 9: print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")
            
            # 反向传播和优化
            optimizer.zero_grad()  # 重要：每次迭代前清零梯度
            loss.backward()
            optimizer.step()
        print('------')

C:\Users\杜骏晔\AppData\Local\Temp\ipykernel_31252\3790411606.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  c_final = torch.tensor(c_final, dtype=torch.float32).view(1,-1)


Epoch [10/10], Loss: 0.15593959391117096
------
Epoch [10/10], Loss: 0.07688181102275848
------
Epoch [10/10], Loss: 0.08950886875391006
------
Epoch [10/10], Loss: 0.10682892799377441
------
Epoch [10/10], Loss: 0.08521535992622375
------
Epoch [10/10], Loss: 0.046864885836839676
------
Epoch [10/10], Loss: 0.0510440431535244
------
Epoch [10/10], Loss: 0.03376225382089615
------
Epoch [10/10], Loss: 0.04148484766483307
------
Epoch [10/10], Loss: 0.028880877420306206
------
Epoch [10/10], Loss: 0.007115131244063377
------
Epoch [10/10], Loss: 0.020038651302456856
------
Epoch [10/10], Loss: 0.005968964193016291
------
Epoch [10/10], Loss: 0.00608148192986846
------
Epoch [10/10], Loss: 0.08919356763362885
------
Epoch [10/10], Loss: 0.07808692008256912
------
Epoch [10/10], Loss: 0.02787322923541069
------
Epoch [10/10], Loss: 0.03176671266555786
------
Epoch [10/10], Loss: 0.03287498652935028
------
Epoch [10/10], Loss: 0.028023412451148033
------
Epoch [10/10], Loss: 0.054505884647

In [13]:
torch.save(model,'result_folder/1thmodel.pth')

In [16]:
np.save('result_folder/1ththeta.npy', theta.detach().numpy())

In [17]:
np.save('result_folder/1thlambda.npy', LAMBDA.detach().numpy())
np.save('result_folder/1thbb.npy', bb.detach().numpy())